In [1]:
import pandas as pd
from pathlib import Path

In [2]:
DATA_PATH = Path('./data/data_original_2017_to_2022.csv')
data = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")
data

/tmp/ipykernel_140941/1661828260.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_PATH, index_col="Unnamed: 0")


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
1,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,...,NaN,NaN,2,41,2,0,2,0,201701,2062
2,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,...,NaN,NaN,6,34,4,0,3,1,201701,2062
3,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,...,NaN,NaN,2,33,6,0,4,2,201701,2062
4,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,...,NaN,NaN,4,43,3,0,3,0,201701,2062
5,2335506,520013,15,NaN,NaN,NaN,M,3,269860000125,1,...,NaN,NaN,4,10,1,0,1,0,201701,2062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206188,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,1,3,3,0,3,0,202211,1244
206189,2383691,522205,001,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,4,10,3,0,3,0,202211,1244
206190,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,2,33,7,0,7,0,202211,1244
206191,2439883,522220,02,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,4,43,2,0,2,0,202211,1244


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206192 entries, 1 to 206192
Data columns (total 28 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   CNES      206192 non-null  int64  
 1   CODUFMUN  206192 non-null  int64  
 2   REGSAUDE  126342 non-null  object 
 3   MICR_REG  11221 non-null   object 
 4   DISTRSAN  122 non-null     float64
 5   DISTRADM  2175 non-null    object 
 6   TPGESTAO  206192 non-null  object 
 7   PF_PJ     206192 non-null  int64  
 8   CPF_CNPJ  206192 non-null  int64  
 9   NIV_DEP   206192 non-null  int64  
 10  CNPJ_MAN  206192 non-null  int64  
 11  ESFERA_A  0 non-null       float64
 12  ATIVIDAD  206192 non-null  int64  
 13  RETENCAO  0 non-null       float64
 14  NATUREZA  0 non-null       float64
 15  CLIENTEL  205097 non-null  float64
 16  TP_UNID   206192 non-null  int64  
 17  TURNO_AT  206192 non-null  int64  
 18  NIV_HIER  0 non-null       float64
 19  TERCEIRO  0 non-null       float64
 20  TP_L

In [4]:
def countNones(df: pd.DataFrame):
    return df.isna().sum()

In [5]:
countNones(data)

CNES             0
CODUFMUN         0
REGSAUDE     79850
MICR_REG    194971
DISTRSAN    206070
DISTRADM    204017
TPGESTAO         0
PF_PJ            0
CPF_CNPJ         0
NIV_DEP          0
CNPJ_MAN         0
ESFERA_A    206192
ATIVIDAD         0
RETENCAO    206192
NATUREZA    206192
CLIENTEL      1095
TP_UNID          0
TURNO_AT         0
NIV_HIER    206192
TERCEIRO    206192
TP_LEITO         0
CODLEITO         0
QT_EXIST         0
QT_CONTR         0
QT_SUS           0
QT_NSUS          0
COMPETEN         0
NAT_JUR          0
dtype: int64